In [3]:
import jams
from AudioPreprocessor import AudioPreprocessor
from DatasetPreprocessor import DatasetPreprocessor
import Settings
import matplotlib.pyplot as plt
import librosa
import librosa.display
import numpy as np

datasetPreprocessor = DatasetPreprocessor()
audioPreprocessor = AudioPreprocessor()

In [9]:
w = 480
h = 320

anno_path = './YOLO/_annotations.txt'
px = 1/plt.rcParams['figure.dpi']  # pixel in inches
dpi = 100

In [20]:
def save_image(filename, audio_chunks, folder):
    fig = plt.figure(frameon=False, figsize=(w/dpi, h/dpi), dpi=dpi)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    img = librosa.display.specshow(librosa.amplitude_to_db(audio_chunks,
                                                        ref=np.max))
    plt.savefig(f'./YOLO/{folder}/{filename}')

def save_anno(filename, anno_file):
    dur = librosa.get_duration(filename = Settings.mic_path + filename + "_mic.wav")
    
    file_anno = Settings.annotations_path + filename + ".jams"
    jam = jams.load(file_anno)

    notes = []
    for string_num in range(6):
        anno = jam.annotations["note_midi"][string_num]
        for note in anno["data"]:
            notes.append([round((note[0] / dur) * w), round(((note[0] + note[1]) / dur) * w)])

    anno_file.write(f'{filename}.png')
    for note in notes:
        anno_file.write(f' {note[0]},0,{note[1]},{h},0')
    anno_file.write('\n')

def process_file(filename, anno_file, folder):
    file_audio = Settings.mic_path + filename + "_mic.wav"
    audio_chunks = audioPreprocessor.process_audiofile(file_audio)
    audio_chunks = np.swapaxes(audio_chunks, 0, 1)

    save_image(filename, audio_chunks, folder)
    save_anno(filename, anno_file)

def split_files(files):
    train_files = []
    test_files = []
    val_files = []
    for i in files:
        result = np.random.choice(3, p=[0.8, 0.1, 0.1])
        if result == 0:
            train_files.append(i)
        elif result == 1:
            test_files.append(i)
        else:
            val_files.append(i)
    return train_files, test_files, val_files

def process_files():
    files = datasetPreprocessor.get_all_filenames()
    train_files, test_files, val_files = split_files(files)
    with open('./YOLO/train/_annotations.txt', 'w') as anno_file:
        for file in train_files:
            process_file(file, anno_file, 'train')
    with open('./YOLO/test/_annotations.txt', 'w') as anno_file:
        for file in test_files:
            process_file(file, anno_file, 'test')
    with open('./YOLO/val/_annotations.txt', 'w') as anno_file:
        for file in val_files:
            process_file(file, anno_file, 'val')        


In [ ]:
process_files()